# CWT & STFT + CNN Model with BCI_competitionIII dataset

In [46]:
import mne
import numpy as np
from mne.datasets import eegbci
from mne.channels import make_standard_montage

# 07
num_subject = 20
raw_RorL1sub = [0]*num_subject
raw_Both1sub = [0]*num_subject
add = 0

for j in range(1+add,num_subject+1+add):

    if j <= 9:
        subject  = '00' + str(j)
    elif j <= 100:
        subject = '0' + str(j)
    else:
        subject = str(j)


    raw_RorL1 = mne.io.read_raw_edf("D:\physionet_dataset\S" + str(subject) +"\S" + str(subject) +"R03.edf",preload = True)
    raw_RorL2 = mne.io.read_raw_edf("D:\physionet_dataset\S" + str(subject) +"\S" + str(subject) +"R07.edf",preload = True)
    raw_RorL3 = mne.io.read_raw_edf("D:\physionet_dataset\S" + str(subject) +"\S" + str(subject) +"R11.edf",preload = True)

    raw_RorL1sub[j-(1+add)] = mne.concatenate_raws([raw_RorL1, raw_RorL2, raw_RorL3])

    raw_Both1 = mne.io.read_raw_edf("D:\physionet_dataset\S" + str(subject) +"\S" + str(subject) +"R05.edf",preload = True)
    raw_Both2 = mne.io.read_raw_edf("D:\physionet_dataset\S" + str(subject) +"\S" + str(subject) +"R09.edf",preload = True)
    raw_Both3 = mne.io.read_raw_edf("D:\physionet_dataset\S" + str(subject) +"\S" + str(subject) +"R13.edf",preload = True)

    raw_Both1sub[j-(1+add)] = mne.concatenate_raws([raw_Both1, raw_Both2, raw_Both3])

# raw_RorL = mne.concatenate_raws(raw_RorL1sub)
# raw_Both = mne.concatenate_raws(raw_Both1sub)

raw_RorL = mne.concatenate_raws([raw_RorL1sub[6]])
raw_Both = mne.concatenate_raws([raw_Both1sub[6]])

# events = mne.read_events("D:\physionet_dataset\S001\S001R02.edf.event")
print(np.shape(raw_RorL.get_data()), np.shape(raw_Both.get_data()))

eegbci.standardize(raw_RorL)  # set channel names
montage = make_standard_montage("standard_1005") 
raw_RorL.set_montage(montage)

eegbci.standardize(raw_Both)  # set channel names
montage = make_standard_montage("standard_1005")
raw_Both.set_montage(montage)

Extracting EDF parameters from C:\git\Senior_Thesis\DataSet\Convert_data\hand0_new.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 279249  =      0.000 ...  1116.996 secs...
Extracting EDF parameters from C:\git\Senior_Thesis\DataSet\Convert_data\hand1_new.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 279249  =      0.000 ...  1116.996 secs...
Extracting EDF parameters from C:\git\Senior_Thesis\DataSet\Convert_data\hand2_new.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 279249  =      0.000 ...  1116.996 secs...
Extracting EDF parameters from C:\git\Senior_Thesis\DataSet\Convert_data\hand3_new.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 279249  =      0.000 ...  1116.996 secs...
Extracting EDF parameters from C:\git\Senior_Thesis\DataSet\Convert_data\hand4_n

<RawEDF | hand3_new.edf, 8 x 279250 (1117.0 s), ~17.1 MB, data loaded>

In [47]:
import numpy as np
import tensorflow as tf
import keras
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda
from keras.models import Model
from keras.models import Sequential
from keras.callbacks import History 
history = History()
import pywt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from ssqueezepy import ssq_cwt, ssq_stft

In [48]:
eeg3 = raw_RorL.copy().filter(l_freq=8, h_freq=30, method = 'iir', iir_params= {"order": 12, "ftype":'butter'})
eeg3 = eeg3.copy().set_eeg_reference(ref_channels="average")
# eeg3= eeg3.pick_channels(['Fz', 'C3', 'Cz', 'C4', 'Pz' ,'PO7', 'Oz', 'PO8'])

eeg4 = raw_Both.copy().filter(l_freq=8, h_freq=30, method = 'iir', iir_params= {"order": 12, "ftype":'butter'})
eeg3 = eeg3.copy().set_eeg_reference(ref_channels="average")
# eeg4= eeg4.pick_channels(['Fz', 'C3', 'Cz', 'C4', 'Pz', 'PO7', 'Oz','PO8'])

eeg3= eeg3.pick_channels(['FC3','C5','CP3','C3','Cz','C4','C6','FC4','CP4', 'Pz', 'PO7', 'Oz','PO8'])
events_RorL, _ = mne.events_from_annotations(eeg3)

eeg4 = eeg4.pick_channels(['FC3','C5','CP3','C3','Cz','C4','C6','FC4','CP4', 'Pz', 'PO7', 'Oz','PO8'])
events_Both, _ = mne.events_from_annotations(eeg4)

event_ids1 = dict(Left=2, Right=3)
event_ids2 = dict(Both_fits=2,Both_feet=3)
# event_ids2 = dict(Both_feet=3)


RorL_epochs = mne.Epochs(eeg3, events_RorL, 
        tmin=-1.5,     # init timestamp of epoch (0 means trigger timestamp same as event start)
        tmax=4.0,    # final timestamp (10 means set epoch duration 10 second)
        event_id= event_ids1,
        preload = True,
        event_repeated='drop'
    )

Both_epochs = mne.Epochs(eeg4, events_Both, 
        tmin=-1.5,     # init timestamp of epoch (0 means trigger timestamp same as event start)
        tmax=4.0,    # final timestamp (10 means set epoch duration 10 second)
        event_id= event_ids2,
        preload = True,
        event_repeated='drop'
    )


CSP_train_RorL = RorL_epochs.copy().crop(tmin=0, tmax=3.5)
labels_RorL = RorL_epochs.events[:, -1] -1 

CSP_train_Both = Both_epochs.copy().crop(tmin=0, tmax=3.5)
labels_Both = Both_epochs.events[:, -1] +1

CSP_train = np.vstack((CSP_train_RorL.get_data(), CSP_train_Both.get_data()))
labels = np.concatenate((labels_RorL,labels_Both))


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 64 (effective, after forward-backward)
- Cutoffs at 8.00, 30.00 Hz: -6.02, -6.02 dB



EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Used Annotations descriptions: ['OVTK_GDF_Cross_On_Screen', 'OVTK_GDF_End_Of_Session', 'OVTK_GDF_End_Of_Trial', 'OVTK_GDF_Feedback_Continuous', 'OVTK_GDF_Incorrect', 'OVTK_GDF_Left', 'OVTK_GDF_Right', 'OVTK_GDF_Start_Of_Trial', 'OVTK_GDF_Tongue', 'OVTK_GDF_Up', 'OVTK_StimulationId_BaselineStart', 'OVTK_StimulationId_BaselineStop', 'OVTK_StimulationId_Beep', 'OVTK_StimulationId_Train']
Multiple event values for single event times found. Keeping the first occurrence and dropping all others.
Not setting metadata
605 matching events found
Setting baseline interval to [-5.0, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 605 events and 2501 original time points ...
2 bad epochs dropped


In [54]:
shape = np.shape(combine_epochs['Left','Right','Both_fits','Both_feet'].get_data())

train_data = np.ndarray(shape=(shape[0], shape[2], shape[1]))
for jj in range(0, shape[0]):
    train_data[jj] = combine_epochs['Left','Right','Both_fits','Both_feet'].get_data()[jj].T
print(np.shape(train_data))

(120, 1001, 3)


In [55]:
train_data.shape[1]

1001

In [56]:
# train_data = combine_epochs['Right','Foot'].get_data().reshape(shape[0], shape[2], shape[1])
print(np.shape(train_data))
scales = range(1,228)

# scales = np.array(scales)
# scales = scales.astype('float32')

waveletname = 'morl'
train_size = len(labels)
train_data_cwt = np.ndarray(shape=(train_size, 227, 1001, 3))

for ii in range(0,train_size):
    if ii % 40 == 0:
        print(ii)
    for jj in range(0,3):
        signal = train_data[ii, :, jj]
        # coeff ,_, *_ = ssq_cwt(signal)
        # _ ,coeff, *_ = ssq_stft(signal, n_fft = 226*2)
        coeff, _ = pywt.cwt(signal, scales, waveletname, 1)
        coeff_ = coeff[:,:1001]  #crop 227 sample for each channel
        train_data_cwt[ii, :, :, jj] = np.abs(coeff_)
print(np.shape(coeff),np.shape(coeff_))

(120, 1001, 3)
0
40
80
(227, 1001) (227, 1001)


In [57]:
print(np.shape(coeff), np.shape(coeff_))

(227, 1001) (227, 1001)


In [58]:
print(np.shape(signal))
print(np.shape(coeff))
print(np.shape(coeff_))
print(np.shape(train_data_cwt))

(1001,)
(227, 1001)
(227, 1001)
(120, 227, 1001, 3)


In [61]:
x_train, x_test, y_train, y_test = train_test_split(train_data_cwt, labels, test_size=0.2, random_state=42)

img_x = 227
img_y =1001
img_z = 3
num_classes = 4

batch_size = 16
epochs = 100
print(np.shape(x_train), np.shape(x_test), np.shape(y_train), np.shape(y_test))

# del train_data_cwt
# del combine_epochs
# plt.imshow(x_train[0,:,:,1], aspect='auto', cmap='turbo')
# plt.show()
# plt.imshow(x_train[2,:,:,1], aspect='auto', cmap='turbo')
# plt.show()
y_train = keras.utils.to_categorical(y_train - 6, num_classes)
y_test = keras.utils.to_categorical(y_test -6 , num_classes)
input_shape = (img_x, img_y, img_z)

(96, 227, 1001, 3) (24, 227, 1001, 3) (96,) (24,)


In [62]:
model = Sequential()
# Layer 1: Convolutional + Max-Pooling
model.add(Conv2D(filters= 96, kernel_size= (11, 11), strides=(4, 4), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((3, 3), strides=(2, 2)))

# Layer 2: Convolutional + Max-Pooling
model.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1) ,activation='relu', padding= 'valid'))
model.add(MaxPooling2D((3, 3), strides=(2, 2), padding= 'valid'))

# Layer 3: Convolutional
model.add(Conv2D(filters =384, kernel_size= (3, 3), strides=(1,1),activation='relu', padding= 'valid'))

# Layer 4: Convolutional
model.add(Conv2D(filters = 384, kernel_size= (3, 3), strides=(1,1),activation='relu', padding= 'valid'))

# Layer 5: Convolutional + Max-Pooling
model.add(Conv2D(filters = 256, kernel_size= (3, 3), strides=(1,1),activation='relu', padding= 'valid'))
model.add(MaxPooling2D((3, 3), strides=(2, 2), padding='valid'))

# Flatten the output for fully connected layers
model.add(Flatten())

# Layer 6: Fully Connected (Dense)
model.add(Dense(4096, activation='relu', input_shape = (13,13,128)))
model.add(keras.layers.Dropout(0.4))

# Layer 7: Fully Connected (Dense)
model.add(Dense(2048, activation='relu'))
model.add(keras.layers.Dropout(0.4))

# Layer 8: Output Layer
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer=keras.optimizers.Adam(learning_rate= 0.0001), 
              metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=batch_size, 
          epochs=epochs, verbose=1, 
          validation_data=(x_test, y_test), 
          callbacks=[history])

Epoch 1/100
6/6 [==============================] - 17s 2s/step - loss: 1.4764 - accuracy: 0.2500 - val_loss: 1.3826 - val_accuracy: 0.2083
Epoch 2/100
6/6 [==============================] - 12s 2s/step - loss: 1.3797 - accuracy: 0.3438 - val_loss: 1.3534 - val_accuracy: 0.2917
Epoch 3/100
6/6 [==============================] - 12s 2s/step - loss: 1.4022 - accuracy: 0.2500 - val_loss: 1.3312 - val_accuracy: 0.4583
Epoch 4/100
6/6 [==============================] - 13s 2s/step - loss: 1.3618 - accuracy: 0.3229 - val_loss: 1.3256 - val_accuracy: 0.3750
Epoch 5/100
6/6 [==============================] - 12s 2s/step - loss: 1.3007 - accuracy: 0.3646 - val_loss: 1.2467 - val_accuracy: 0.3333
Epoch 6/100
6/6 [==============================] - 12s 2s/step - loss: 1.2380 - accuracy: 0.4062 - val_loss: 1.4282 - val_accuracy: 0.2917
Epoch 7/100
6/6 [==============================] - 12s 2s/step - loss: 1.2005 - accuracy: 0.4583 - val_loss: 1.2388 - val_accuracy: 0.3333
Epoch 8/100
6/6 [==========

In [ ]:
train_score = model.evaluate(x_train, y_train, verbose=0)
print('Train loss: {}, Train accuracy: {}'.format(train_score[0], train_score[1]))
test_score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss: {}, Test accuracy: {}'.format(test_score[0], test_score[1]))

Train loss: 0.6932258009910583, Train accuracy: 0.4933035671710968
Test loss: 0.6930710673332214, Test accuracy: 0.5089285969734192
